In [1]:
"""
    Ranking:
        1.1 Hierarchical Clustering (ranking_prediction.ipynb)
        1.2 Map labels to future matches as home and away of past season
            1.2.1 If team is new, label it as normal (which is a not promoted team)
        1.3 Compute difference in ranking (NO because of no sequential)
        1.4 Future work: study the week development of the league to see if there is a point where
            it can be started to predict the actual behavior of the team as the promotion label
    Prediction:
        2.0 Drop features --> computation differences (visualization.ipynb)
        2.1 Features TEST
            2.1.0 First match as median of past season (already started) TODO
            2.1.2 Median of current season (home and away games are definetely a factor)
            2.1.2.1 Median at home/away <-- Just this
            2.1.2.2 Past average of last n games (test with 3, 5, etc)
            2.1.2.3 Average of last n games at home/away
"""

'\n    Ranking:\n        1.1 Hierarchical Clustering (ranking_prediction.ipynb)\n        1.2 Map labels to future matches as home and away of past season\n            1.2.1 If team is new, label it as normal (which is a not promoted team)\n        1.3 Compute difference in ranking (NO because of no sequential)\n        1.4 Future work: study the week development of the league to see if there is a point where\n            it can be started to predict the actual behavior of the team as the promotion label\n    Prediction:\n        2.0 Drop features --> computation differences (visualization.ipynb)\n        2.1 Features TEST\n            2.1.0 First match as median of past season (already started) TODO\n            2.1.2 Median of current season (home and away games are definetely a factor)\n            2.1.2.1 Median at home/away <-- Just this\n            2.1.2.2 Past average of last n games (test with 3, 5, etc)\n            2.1.2.3 Average of last n games at home/away\n'

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import playstyle
import pickle
import ranking

In [2]:
"""
    Everything to create the clusters
"""
from sklearn import preprocessing
from scipy.cluster.hierarchy import linkage, cophenet, fcluster
from scipy.spatial.distance import pdist

# Returns Z, coph_matrix and best cophence score from HC
def HierarchicalClustering(data, label):
    methods = ["single","complete","average","centroid","ward"]

    # Pass the dataset into pdist to get your proximity matrix for calculating CPCC
    proximity_matrix = pdist(data)

    best_coph = -1
    best_method = None

    for method in methods:
        Z = linkage(data, method)
        coph, coph_matrix = cophenet(Z, proximity_matrix)
        if coph > best_coph:
            best_coph = coph
            best_method = method
            best_matrix = coph_matrix
        Z = linkage(data, best_method)
        coph_matrix = cophenet(Z)
    return Z, coph_matrix, best_coph

#Returns a dictionary with the clusters in the form
#"id": "season" : point -> Attr object
def dct_clusters(dct_clusters, Z, coph_matrix, dendo_label, criterion='distance', real=None):
    if real != None:
        clusters = real[0]
    else:
        clusters = fcluster(Z, t=coph_matrix[0], criterion=criterion)
    for i in range(0, len(dendo_label)):
        point = ranking.Attr(dendo_label[i], clusters[i])
        if point.value2 not in dct_clusters:
            dct_clusters[point.value2] = {}
            dct_clusters[point.value2][point.value3] = point
        else:
            dct_clusters[point.value2][point.value3] = point
    return dct_clusters, clusters

# Return the cluster of an specific type
def get_cluster_of_type(all_dct, country, league, seasons, target_col, clean_type, real=None):
    # Data initialization
    data, column_names = ranking.concat_data(country, league, seasons, target_col, clean_type)
    all_data, all_season, all_names, all_target = ranking.get_all_data(data, len(target_col))
    # Use Agglomerative
    all_data = preprocessing.StandardScaler().fit_transform(all_data)
    dendo_label = ranking.label_team_season(all_names, all_season)
    Z, coph_matrix, coph = HierarchicalClustering(all_data, dendo_label)
    if real == None:
        all_dct, all_lst = dct_clusters(all_dct, Z, coph_matrix, dendo_label)
    else:
        all_dct, all_lst = dct_clusters(all_dct, Z, coph_matrix, dendo_label, real=all_target)
    return all_dct

#Returns three clusters as: overall, home and away performance
def get_clusters(country, league, curr_week, season, year_window=1):
    # Historicity: multi season directory
    start_season = season
    seasons = list(range(start_season, start_season - year_window, -1))
    seasons = sorted(seasons, reverse=True)
    # Targets
    target_col = ["rank", "points", "description"]
    # Gets the dictionary
    overall_dct = dict()
    home_dct = dict()
    away_dct = dict()
    real_dct = dict()
    for season in seasons:
        season = [season]
        real_dct = get_cluster_of_type(overall_dct, country, league, season, target_col, clean_type=None, real=True)
        overall_dct = get_cluster_of_type(overall_dct, country, league, season, target_col, clean_type=None)
        home_dct = get_cluster_of_type(home_dct, country, league, season, target_col, clean_type='home')
        away_dct = get_cluster_of_type(away_dct, country, league, season, target_col, clean_type='away')
    return (overall_dct, home_dct, away_dct, real_dct)

In [3]:
"""
    Creates the statistics dataset
"""
def get_statistics(country, league, curr_week, season, year_window=1):
    # Historicity: multi season directory
    start_season = season
    seasons = list(range(start_season, start_season - year_window, -1))
    seasons = sorted(seasons, reverse=True)
    # Data initialization
    data = []
    for season in seasons:
        df, target = playstyle.df_season(country, league, season, curr_week, drop_goals=False)
        tup = (season, df, target)
        data.append(tup)
    all_data, all_target = playstyle.get_all(data)
    return all_data, all_target

In [4]:
"""
    Function that maps the dataset with the clusters
"""
def max_appearance_rank(dct, season):
    dct_rank = dict()
    season = str(int(season))
    for team in dct:
        if season in dct[team]:
            rank = dct[team][season].value
            if rank not in dct_rank:
                dct_rank[rank] = 1
            else:
                dct_rank[rank] += 1
    return max(dct_rank.keys(), key=dct_rank.get)

def get_rank(df, dct, team):
    rank = []
    for index, row in df.iterrows():
        season = row["season"]
        season = season - 1
        if str(row[team]) in dct:
            if str(season) in dct[str(row[team])]:
                rank.append(dct[str(row[team])][str(season)].value)
            else:
                max_appear = max_appearance_rank(dct, season)
                rank.append(max_appear)
        else:
            max_appear = max_appearance_rank(dct, season)
            rank.append(max_appear)
    return rank

def get_data(dct, statistics):
    statistics["home_team.overall_rank"] = get_rank(statistics, dct[0], "home_team.id")
    statistics["away_team.overall_rank"] = get_rank(statistics, dct[0], "away_team.id")
    statistics["home_team.ranking"] = get_rank(statistics, dct[1], "home_team.id")
    statistics["away_team.ranking"] = get_rank(statistics, dct[2], "away_team.id")
    statistics["home_team.real_rank"] = get_rank(statistics, dct[3], "home_team.id")
    statistics["away_team.real_rank"] = get_rank(statistics, dct[3], "away_team.id")
    return statistics, statistics["home_team.overall_rank"].values.reshape(-1, 1), statistics["away_team.overall_rank"].values.reshape(-1, 1), statistics["home_team.ranking"].values.reshape(-1, 1), statistics["away_team.ranking"].values.reshape(-1, 1),statistics["home_team.real_rank"].values.reshape(-1, 1), statistics["away_team.real_rank"].values.reshape(-1, 1)

In [5]:
def get_median(df, method=None, season=None, week=None):
    # GET median for team_home.stats_home. team_home.stats_away. team_away.stats_home. team_away.stats_away.
    # Filter df according to method
    if method == '1':
        print("Current and past season median")
        df = df[(df["season"].isin([season, season-1]))]
    elif method == '2':
        print("Current and all past seasons median")
        df = df[(df["season"] <= season)]
    elif method == '3':
        print("Current season only median")
        # First and second week zero/one home/away games
        if week > 3:
            df = df[(df["season"] == season)]
        else:
            df = df[(df["season"].isin([season, season-1]))]
    elif method == '4':
        print("Last 5 games median")
        if week > 5:
            weeks = [w for w in range(week-1, week-6, -1)]
            df = df[(df["season"] == season) & (df["week"].isin(weeks))]
        else:
            df = df[(df["season"].isin([season, season-1]))]
        
    # Median home
    extra_columns = df.filter(["goals_home"]).columns
    columns_home = df.filter(regex='^stats_home').columns
    columns_home = columns_home.append(extra_columns)
    median_home = df.groupby(['home_team.id'], as_index=True)[columns_home].median()
    # Median away
    extra_columns = df.filter(["goals_away"]).columns
    columns_away = df.filter(regex='^stats_away').columns
    columns_away = columns_away.append(extra_columns)
    median_away = df.groupby(['away_team.id'], as_index=True)[columns_away].median()
    if method == '5':
        print("Current season only mean")
        # First and second week zero/one home/away games
        if week > 3:
            df = df[(df["season"] == season)]
        else:
            df = df[(df["season"].isin([season, season-1]))]
        mean_home = df.groupby(['home_team.id'], as_index=True)[columns_home].mean()
        mean_away = df.groupby(['away_team.id'], as_index=True)[columns_away].mean()
        return mean_home, mean_away
    return median_home, median_away

In [6]:
def create_test_set(df, teams, season, week, pezzali=True, method=None, extras=None):
    columns_home = ["goals_home", "stats_home.s_off_g", "stats_home.s_on_g", "stats_home.s_in", 
                   "stats_home.saves", "stats_home.s_blocked", "stats_home.c_yellow", "stats_home.s_out"]
    columns_away = ["goals_away", "stats_away.s_off_g", "stats_away.s_on_g", "stats_away.s_in", 
                   "stats_away.saves", "stats_away.s_blocked", "stats_away.c_yellow", "stats_away.s_out"]
    columns = ["season", "week"]
    columns = columns_home + columns_away + columns
    columns_pezzali = columns
    
    if pezzali == False or method == '5' or extras == '4' or extras == '3':
        extra_columns = df.filter(["goals_home"]).columns
        columns_home = df.filter(regex='^stats_home').columns
        columns_home = columns_home.append(extra_columns)
        extra_columns = df.filter(["goals_away"]).columns
        columns_away = df.filter(regex='^stats_away').columns
        columns_away = columns_away.append(extra_columns)
        columns = columns_home.append(columns_away)
    
    test_set = pd.DataFrame(columns=columns)
    home_teams = [match[0] for match in teams]
    away_teams = [match[1] for match in teams]
    seasons = [season for i in range(0, len(teams))]
    weeks = [week for i in range(0, len(teams))]
    test_set["home_team.id"] = home_teams
    test_set["away_team.id"] = away_teams
    test_set["season"] = seasons
    test_set["week"] = weeks
    
    columns_home = test_set.filter(columns_home).columns
    columns_away = test_set.filter(columns_away).columns
    i_th = test_set.columns.get_loc("home_team.id")
    i_ta = test_set.columns.get_loc("away_team.id")
    
    median_home, median_away = get_median(df, method, season, week)

    for i in range(0, len(test_set)):
        for index, row in median_home.iterrows():
            if test_set.iloc[i,i_th] == index:
                for c in columns_home:
                    try:
                        i_c = test_set.columns.get_loc(c)
                        test_set.iloc[i,i_c] = row[c]
                    except:
                        print(index, c)
    for i in range(0, len(test_set)):
        for index, row in median_away.iterrows():
            if test_set.iloc[i,i_ta] == index:
                for c in columns_away:
                    try:
                        i_c = test_set.columns.get_loc(c)
                        test_set.iloc[i,i_c] = row[c]
                    except:
                        print(index, c)
    # For non existing teams (this means ascending teams fil with median of column)
    test_set = test_set.apply(lambda x: x.fillna(x.median()),axis=0)
    if method == '5':
        test_set = linear_reg(test_set, columns)
        if pezzali == True:
            return test_set[['week','season','away_team.id', 'home_team.id', 'goals_home', 'stats_home.s_off_g', 'stats_home.s_on_g', 'stats_home.s_in', 'stats_home.saves', 'stats_home.s_blocked', 'stats_home.c_yellow', 'stats_home.s_out', 'goals_away', 'stats_away.s_off_g', 'stats_away.s_on_g', 'stats_away.s_in', 'stats_away.saves', 'stats_away.s_blocked', 'stats_away.c_yellow', 'stats_away.s_out']]
    return test_set

In [7]:
def target_no_negative(target):
    # Target must be nonegative
    for i in range(0, len(target)):
        if target[i] < 0:
            target[i] = 2
    return target

In [8]:
import random
from scipy.stats import chi2_contingency 
def chi_squared_clusters(df, target):
    columns = ["home_team.ranking", "away_team.ranking","home_team.overall_rank", "away_team.overall_rank","home_team.real_rank", "away_team.real_rank"]
    # Target must be nonegative
    target = target_no_negative(target)
    # Get daframe randoms
    indexes = [random.randint(0, 380) for x in range(0, 100)]
    for c in columns:
        col1 = [df[c].values[indexes[i]] for i in range(0, 100)]
        #col2 = [df[c[1]].values[indexes[i]] for i in range(0, 100)]
        test_target = [target[indexes[i]] for i in range(0, 100)]
        #diff = np.absolute(np.array(col1)-np.array(col2))
        data = [col1, test_target]
        stat, p, dof, expected = chi2_contingency(data)
        # interpret p-value 
        alpha = 0.05
        if p <= alpha: 
            print('Significant relation c = %s, p = %f' %(c, p)) 
        else: 
            print('Not significant c = %s, p = %f' %(c, p)) 

In [9]:
def linear_reg(test_set, columns):
    for i in range(0, len(test_set)):
        for c in columns:
            try:
                i_c = test_set.columns.get_loc(c)
                test_set.iloc[i,i_c] = get_regression(test_set.iloc[i], c)
            except:
                print(i, c)
                raise
    return test_set

In [10]:
def get_regression(row, column):
    global country
    # Load model
    filename = 'model/' + country + '/' + column + '.sav'
    model = pickle.load(open(filename, 'rb'))
    row = row.drop(column)
    row = row.drop("home_team.id")
    row = row.drop("away_team.id")
    row = row.drop("season")
    row = row.drop("week")  
    pred = np.array(row).reshape(1, -1)
    return model.predict(pred)

In [11]:
# Return the index for the confusion matrix
def get_index(label):
    if label == -1:
        return 2
    else:
        return label

In [12]:
def balance_dataset(dataset, target, draw=False):
    if draw:
        dataset = pd.DataFrame(dataset)
        dataset["target"] = target
        dataset = dataset[(dataset["target"] != 1)]
        new_target = np.array(dataset["target"])
        dataset = dataset.drop(columns=["target"], axis=1)
        for i in range(0, len(new_target)):
            if new_target[i] == 0:
                new_target[i] = 1
            elif new_target[i] == -1:
                new_target[i] = 0
        return dataset, new_target
    else:
        new_target = []
        for i in range(0, len(target)):
            if target[i] != 1:
                new_target.append(0)
            else:
                new_target.append(1)
        return dataset, new_target

In [13]:
"""
    A proper evaluation: we cannot test week 10 2018 with week 20 2018
    The sequence of events must be maintained
"""
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
 
def subtraining_testset(df, season, week):
    df_sub = df[(df["season"] == season) & (df["week"] < week)]
    
    if week < 3:
        df_past = df[(df["season"] == season-1)]
        df_sub = df_sub.append(df_past)
    
    return df_sub

def subtraining_trainset(df, season, week, up_to_season):
    df_sub = df[(df["season"] == season) & (df["week"] < week)]
    target = []
    
    past_seasons = [s for s in range(up_to_season, season)]
    
    df_past = df[(df["season"].isin(past_seasons))]
    
    df_sub = df_sub.append(df_past)
    
    for index, row in df_sub.iterrows():
        target.append(playstyle.get_status(row))
    
    return df_sub, target

def test_list(df, season, week):
    df_test = df[(df["season"] == season) & (df["week"] == week)]
    teams = []
    target = []
    for index, row in df_test.iterrows():
        teams.append([row["home_team.id"], row["away_team.id"]])
        target.append(playstyle.get_status(row))
    return teams, target


In [14]:
# Data found to be not relevant to models
# Now it is replaced by Anova pipeline
def remove_before_models(df, extras):
    if extras != '1':
        try:
            df = df.drop(columns=['index'], axis=1)
        except:
            print("")
    return df

In [15]:
# Data found to be not relevant to models
# If not using Anova
def remove_before_models(df, extras):
    if extras != '1':
        try:
            df = df.drop(columns=['stats_home.p_accurate','stats_away.p_accurate','stats_away.p_total','stats_away.fouls','stats_home.p_total','stats_home.fouls','stats_home.corners','stats_home.offside','stats_away.corners','stats_away.offside'], axis=1)
        except:
            print("NO")
        try:
            df = df.drop(columns=['index'], axis=1)
        except:
            print("")
    return df

In [16]:
def models_evaluation(train_set, target, test_set, test_target, accuracy_dct, total_tests, k_best=10):
    total_tests = total_tests + len(test_target)
    print(total_tests)
    st = preprocessing.StandardScaler().fit_transform(train_set)
    sp = preprocessing.StandardScaler().fit_transform(test_set)
    # 1) anova filter, take n best ranked features
    anova_filter = SelectKBest(f_regression, k=k_best)
    classifiers = dict(nb=GaussianNB(), svc=SVC(kernel='linear', probability=True), lreg=linear_model.LogisticRegression(max_iter=1000), rforest=RandomForestClassifier(criterion="entropy", max_features="log2", min_samples_leaf=2))
    for name, clf in classifiers.items():
        #anova_model = make_pipeline(anova_filter, clf)
        anova_model = clf
        victory_dataset, victory_target = balance_dataset(st, target)
        draw_dataset, draw_target = balance_dataset(st, target, draw=True)
        # WITH VICTORIES
        anova_model.fit(victory_dataset, victory_target)
        Y_victory = anova_model.predict(sp)
        # WITH DRAWS
        anova_model.fit(draw_dataset, draw_target)
        Y_draw = anova_model.predict(sp)
        Y = []
        for i in range(0, len(Y_victory)):
            if Y_victory[i] == 0:
                if Y_draw[i] == 1:
                    Y.append(0)
                # Not a draw, then a loose
                else:
                    Y.append(-1)
            else:
                Y.append(1)
        #anova_model.fit(st, target)
        #Y = anova_model.predict(sp)
        #print(name)
        #print(Y)
        #print(test_target)
        #print(accuracy_score(test_target, Y, normalize=False))
        for i in range(0, len(test_target)):
            i_predicted = get_index(Y[i])
            i_actual = get_index(test_target[i])
            accuracy_dct[name]["predicted"][i_actual][i_predicted] += 1
        accuracy_dct[name]["score"] += accuracy_score(test_target, Y, normalize=False)
    return accuracy_dct, total_tests

In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import RidgeClassifierCV

"""
    Multilabel learning with sci-kit
"""
def models_evaluation(train_set, target, test_set, test_target, accuracy_dct, total_tests, k_best=10):
    total_tests = total_tests + len(test_target)
    print(total_tests)
    st = preprocessing.StandardScaler().fit_transform(train_set)
    sp = preprocessing.StandardScaler().fit_transform(test_set)
    # 1) anova filter, take n best ranked features
    anova_filter = SelectKBest(f_regression, k=k_best)
    classifiers = dict(nb_ovo=OneVsOneClassifier(GaussianNB()),
                       svc_ovo=OneVsOneClassifier(SVC(kernel='linear', probability=True)),
                       ridge=OneVsOneClassifier(RidgeClassifierCV(class_weight='balanced')), 
                       rforest=OneVsOneClassifier(RandomForestClassifier(random_state=0, criterion="entropy", max_features="log2", min_samples_leaf=2)))
    for name, clf in classifiers.items():
        anova_model = make_pipeline(anova_filter, clf)
        #anova_model = clf
        anova_model.fit(st, target)
        Y = anova_model.predict(sp)
        #print(name)
        #print(Y)
        #print(test_target)
        #print(accuracy_score(test_target, Y, normalize=False))
        for i in range(0, len(test_target)):
            i_predicted = get_index(Y[i])
            i_actual = get_index(test_target[i])
            accuracy_dct[name]["predicted"][i_actual][i_predicted] += 1
        accuracy_dct[name]["score"] += accuracy_score(test_target, Y, normalize=False)
    return accuracy_dct, total_tests

In [18]:
"""
    This function returns a training set with the given method
"""
def train_method(dct, train_set, s, w, seasons, method, extras=None):
    print("Train Method")
    statistics_subset = None
    target = None
    if method == '1':
        print("Current and past season median")
    elif method == '2':
        print("Current and all past seasons median")
    elif method == '3':
        print("Current season only median")
    elif method == '4':
        print("Last 5 games median")
    elif method == '5':
        print("Current and past season linear regression")
    elif method == '6':
        print("Current and all past seasons linear regression")
    elif method == '7':
        print("Current season only linear regression")
    elif method == '8':
        print("Plain train set: all games from past seasons")
        statistics_subset, target = subtraining_trainset(train_set, s, w, seasons)
   
    # If something must be removed
    statistics_removed = remove_before_models(statistics_subset, extras)
    n = 10
    if extras == '1':
        print("Applies pezzali to train set")
        statistics_subset = playstyle.pezzali_data(statistics_subset, is_train=True)
        statistics_subset,_,_,_,_,_,_ = get_data(dct, statistics_subset)
        statistics_subset = statistics_subset.drop(columns=['home_team.name', 'away_team.name', 'season', 'week'])
    elif extras == '2':
        print("Applies pca to train set")
        statistics_subset,ho,ao,hr,ar,rh,ra = get_data(dct, statistics_removed)
        statistics_subset = statistics_subset.drop(columns=['home_team.id', 'away_team.id', 'home_team.name', 'away_team.name', 'goals_away','goals_home','season','week'])
        pca = PCA(n_components=n)
        statistics_subset = pca.fit_transform(statistics_subset)
        statistics_subset = np.append(statistics_subset, ho, axis=1)
        statistics_subset = np.append(statistics_subset, ao, axis=1)
        statistics_subset = np.append(statistics_subset, hr, axis=1)
        statistics_subset = np.append(statistics_subset, ar, axis=1)
        statistics_subset = np.append(statistics_subset, rh, axis=1)
        statistics_subset = np.append(statistics_subset, ra, axis=1)
    elif extras == '3':
        print("Adds pezzali and then pca to train set")
        statistics_subset = playstyle.pezzali_data(statistics_removed, is_train=True, both=True)
        statistics_subset,ho,ao,hr,ar,rh,ra = get_data(dct, statistics_subset)
        statistics_subset = statistics_subset.drop(columns=['home_team.id', 'away_team.id', 'home_team.name', 'away_team.name', 'goals_away','goals_home','season','week'])
        pca = PCA(n_components=n)
        statistics_subset = pca.fit_transform(statistics_subset)
        statistics_subset = np.append(statistics_subset, ho, axis=1)
        statistics_subset = np.append(statistics_subset, ao, axis=1)
        statistics_subset = np.append(statistics_subset, hr, axis=1)
        statistics_subset = np.append(statistics_subset, ar, axis=1)
        statistics_subset = np.append(statistics_subset, rh, axis=1)
        statistics_subset = np.append(statistics_subset, ra, axis=1)
    elif extras == '4':
        print("Adds pezzali to plain train set")
        statistics_subset = playstyle.pezzali_data(statistics_removed, is_train=True, both=True)
        statistics_subset,ho,ao,hr,ar,rh,ra = get_data(dct, statistics_removed)
        statistics_subset = statistics_subset.drop(columns=['home_team.id', 'away_team.id', 'home_team.name', 'away_team.name', 'goals_away','goals_home','season','week'])
    elif extras == '0':
        print("Plain train set with clusters")
        statistics_subset,_,_,_,_,_,_ = get_data(dct, statistics_removed)
        statistics_subset = statistics_subset.drop(columns=['home_team.id', 'away_team.id','home_team.name', 'away_team.name','goals_away','goals_home','season','week'])
    return statistics_subset, target

In [19]:
"""
    This function returns a test set with the given method
"""
def test_method(dct, test_set, s, w, method, extras=None):
    print("Test Method")
    teams, test_target = test_list(test_set, s, w)
    if extras == '2' or extras == '0':
        p = False
    else:
        p = True
    if 1 <= int(method) <= 4:
        statistics_test = create_test_set(test_set, teams, s, w, p, method, extras)
    elif method == '5':
        print("Current season only linear regression")
        statistics_test = create_test_set(test_set, teams, s, w, p, method)
    
    # If something must be removed
    statistics_removed = remove_before_models(statistics_test, extras)
    n = 10
    if extras == '1':
        print("Applies pezzali to test set")
        # Get test pezalli and mappings
        statistics_test = playstyle.pezzali_data(statistics_test, is_train=False)
        statistics_test,_,_,_,_,_,_ = get_data(dct, statistics_test)
        try:
            statistics_test = statistics_test.drop(columns=['season','week'])
        except:
            print("")
    elif extras == '2':
        print("Applies pca to test set")
        statistics_test,ho,ao,hr,ar,rh,ra = get_data(dct, statistics_test)
        pca = PCA(n_components=n)
        statistics_test = pca.fit_transform(statistics_removed)
        statistics_test = np.append(statistics_test, ho, axis=1)
        statistics_test = np.append(statistics_test, ao, axis=1)
        statistics_test = np.append(statistics_test, hr, axis=1)
        statistics_test = np.append(statistics_test, ar, axis=1)
        statistics_test = np.append(statistics_test, rh, axis=1)
        statistics_test = np.append(statistics_test, ra, axis=1)
    elif extras == '3':
        print("Adds pezzali and then pca to test set")
        statistics_removed = playstyle.pezzali_data(statistics_removed, is_train=False, both=True)
        statistics_removed = statistics_removed.drop(columns=['home_team.id', 'away_team.id', 'goals_away','goals_home','season','week'])
        statistics_test,ho,ao,hr,ar,rh,ra = get_data(dct, statistics_test)
        pca = PCA(n_components=n)
        statistics_test = pca.fit_transform(statistics_removed)
        statistics_test = np.append(statistics_test, ho, axis=1)
        statistics_test = np.append(statistics_test, ao, axis=1)
        statistics_test = np.append(statistics_test, hr, axis=1)
        statistics_test = np.append(statistics_test, ar, axis=1)
        statistics_test = np.append(statistics_test, rh, axis=1)
        statistics_test = np.append(statistics_test, ra, axis=1)
    elif extras == '4':
        print("Adds pezzali to plain test set")
        statistics_removed = playstyle.pezzali_data(statistics_removed, is_train=False, both=True)
        statistics_removed = statistics_removed.drop(columns=['home_team.id', 'away_team.id', 'goals_away','goals_home','season','week'])
        statistics_test,ho,ao,hr,ar,rh,ra = get_data(dct, statistics_test)
        statistics_test = np.append(statistics_removed, ho, axis=1)
        statistics_test = np.append(statistics_test, ao, axis=1)
        statistics_test = np.append(statistics_test, hr, axis=1)
        statistics_test = np.append(statistics_test, ar, axis=1)
        statistics_test = np.append(statistics_test, rh, axis=1)
        statistics_test = np.append(statistics_test, ra, axis=1)
    elif extras == '0':
        print("Plain test set with clusters")
        statistics_test,_,_,_,_,_,_ = get_data(dct, statistics_removed)
        statistics_test = statistics_test.drop(columns=['home_team.id', 'away_team.id','goals_away','goals_home','season','week'])
    return statistics_test, test_target

In [20]:
# This test is when testing against current season and previous seasons
def custom_test(clusters_dct, train_m, test_m, df, season, year_window=1, week=39, extras=None, k_best=10):
    # Historicity: multi season directory
    start_season = season
    seasons = list(range(start_season, start_season - year_window, -1))
    seasons.reverse()
    # First row draws, second wins, third loses
    # Rows actual vs columns predicted
    accuracy_dct = {
        "nb":{
            "score":0,
            "predicted":[[0,0,0],[0,0,0],[0,0,0]]
        },
        "lreg":{
            "score":0,
            "predicted":[[0,0,0],[0,0,0],[0,0,0]]
        },
        "svc":{
            "score":0,
            "predicted":[[0,0,0],[0,0,0],[0,0,0]]
        },
        "rforest":{
            "score":0,
            "predicted":[[0,0,0],[0,0,0],[0,0,0]]
        },
        "nb_ovo":{
            "score":0,
            "predicted":[[0,0,0],[0,0,0],[0,0,0]]
        },
        "svc_ovo":{
            "score":0,
            "predicted":[[0,0,0],[0,0,0],[0,0,0]]
        },
        "ridge":{
            "score":0,
            "predicted":[[0,0,0],[0,0,0],[0,0,0]]
        }
    }
    total_tests = 0
    for i in range(1, len(seasons)):
        # Test against 38 weeks
        s = seasons[i]
        if s != season:
            up_to_week = 39
        else:
            up_to_week = week
        for w in range(1, up_to_week):
            statistics_training, target = train_method(clusters_dct, df, s, w, seasons[0], train_m, extras)
            statistics_predict, test_target = test_method(clusters_dct, df, s, w, test_m, extras)
            accuracy_dct, total_tests = models_evaluation(statistics_training, target, statistics_predict, test_target, accuracy_dct, total_tests, k_best)
    
    for clf in accuracy_dct:
        accuracy_dct[clf]["score"] = accuracy_dct[clf]["score"]/total_tests
    
    return accuracy_dct

In [21]:
"""
    Everything to retrieve the data
"""
global country
country = 'ES'
league = '140'
curr_week = 38
season = 2019

# Dictionary for the clusters (clusters from 2016 - 2019)
dct = get_clusters(country, league, curr_week, season, year_window=4)

# Dataframe for the match (test from 2017 to 2019)
statistics_to_test, target = get_statistics(country, league, curr_week, season, year_window=3)

In [145]:
# Chi-squared test for clusters
chi_df,_,_,_,_,_,_ = get_data(dct, statistics_to_test)
chi_squared_clusters(chi_df, target)

Not significant c = home_team.ranking, p = 0.998377
Not significant c = away_team.ranking, p = 0.999541
Not significant c = home_team.overall_rank, p = 1.000000
Not significant c = away_team.overall_rank, p = 1.000000
Not significant c = home_team.real_rank, p = 1.000000
Not significant c = away_team.real_rank, p = 1.000000


In [71]:
# Dataframe for the match (test from 2017 to 2019)
up_to_season = 2019
year_window = 3 #2019 + 1
accuracy = custom_test(dct, '8', '3', statistics_to_test, up_to_season, year_window=year_window, extras='0')
print(accuracy)

Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
10
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
20
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
30
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
40
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
50
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
60
Train Method
Plain train set: all 

Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
520
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
530
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
540
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
550
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
560
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
570
Train Method
Plain train set

In [76]:
# Dataframe for the match (test from 2017 to 2019) NOT ANOVA
up_to_season = 2019
year_window = 3 #2019 + 1
accuracy = custom_test(dct, '8', '3', statistics_to_test, up_to_season, year_window=year_window, extras='4')
print(accuracy)

Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
10
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
20
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
30
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
40
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
50
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
60
Train Meth

Test Method
Current season only median

Adds pezzali to plain test set
500
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
510
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
520
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
530
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
540
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
550
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test

In [84]:
# Dataframe for the match (test from 2017 to 2019)
up_to_season = 2019
year_window = 3 #2019 + 1
accuracy = custom_test(dct, '8', '3', statistics_to_test, up_to_season, year_window=year_window, extras='0')
print(accuracy)

Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
10
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
20
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
30
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
40
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
50
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
60
Train Method
Plain train set: all 


Plain test set with clusters
510
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
520
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
530
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
540
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
550
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
560
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with cluster

In [89]:
# Dataframe for the match (test from 2017 to 2019) NOT ANOVA
up_to_season = 2019
year_window = 3 #2019 + 1
accuracy = custom_test(dct, '8', '3', statistics_to_test, up_to_season, year_window=year_window, extras='0')
print(accuracy)

Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
10
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
20
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
30
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
40
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
50
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
60
Train Method
Plain train set: all 


Plain test set with clusters
510
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
520
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
530
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
540
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
550
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
560
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with cluster

In [123]:
# Dataframe for the match (test from 2017 to 2019)
# Multilabel Scikit
# NOT ANOVA
up_to_season = 2019
year_window = 3 #2019 + 1
accuracy = custom_test(dct, '8', '3', statistics_to_test, up_to_season, year_window=year_window, extras='0')
print(accuracy)

Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
10
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
20
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
30
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
40
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
50
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
60
Train Method
Plain train set: all 

Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
520
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
530
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
540
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
550
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
560
Train Method
Plain train set: all games from past seasons

Plain train set with clusters
Test Method
Current season only median

Plain test set with clusters
570
Train Method
Plain train set

In [131]:
# Dataframe for the match (test from 2017 to 2019)
# Multilabel Scikit
# With ANOVA
up_to_season = 2019
year_window = 3 #2019 + 1
accuracy = custom_test(dct, '8', '3', statistics_to_test, up_to_season, year_window=year_window, extras='4')
print(accuracy)

Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
10
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
20
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
30
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
40
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
50
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
60
Train Meth

Test Method
Current season only median

Adds pezzali to plain test set
500
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
510
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
520
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
530
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
540
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test Method
Current season only median

Adds pezzali to plain test set
550
Train Method
Plain train set: all games from past seasons

Adds pezzali to plain train set
Test

In [ ]:
"""
    TODOs
    Test set: checar mean, checar mean/median en un intervalo de 5 partidos
    Train set: checar todas las diferencias, 
"""

In [10]:
# Testing set to predict
# Creating fake records
teams = [[545, 538], [715, 531], [724, 727], [542, 543], [720, 548], [533, 726], [532, 539]]
season = 2020
week = 1
statistics_test = create_test_set(statistics_to_test, teams, season, week)
# Calculating pezzali score
test_pezzali = playstyle.pezzali_data(statistics_test, is_train=False)
# 1. Predicting with clusters of last season
test_pezzali.head()

,home_team.id,away_team.id,diff_pezzali,diff_s_fraction,diff_defensive,stats_away.c_yellow,stats_home.c_yellow,season,week
0,545,538,-0.507937,-0.096591,-3.0,2.0,3.0,2020,1
1,715,531,-0.647727,0.023810,1.0,2.0,2.0,2020,1
2,724,727,-0.236111,0.018519,0.5,3.0,2.0,2020,1
3,542,543,0.236111,0.046154,1.0,3.0,3.0,2020,1
4,720,548,-0.267857,0.000000,-1.0,2.0,2.0,2020,1


In [11]:
statistics_predict = get_data(dct, test_pezzali)

In [12]:
statistics_predict

,home_team.id,away_team.id,diff_pezzali,diff_s_fraction,diff_defensive,stats_away.c_yellow,stats_home.c_yellow,season,week,home_team.overall_rank,away_team.overall_rank,home_team.ranking,away_team.ranking
0,545,538,-0.507937,-0.096591,-3.0,2.0,3.0,2020,1,3,3,4,2
1,715,531,-0.647727,0.023810,1.0,2.0,2.0,2020,1,3,3,4,2
2,724,727,-0.236111,0.018519,0.5,3.0,2.0,2020,1,3,3,4,2
3,542,543,0.236111,0.046154,1.0,3.0,3.0,2020,1,3,3,4,2
4,720,548,-0.267857,0.000000,-1.0,2.0,2.0,2020,1,3,3,4,2
5,533,726,0.174242,0.033333,0.5,2.0,2.0,2020,1,3,3,4,2
6,532,539,0.583333,-0.010989,-1.0,2.0,1.0,2020,1,3,3,4,2


In [14]:
st = remove_before_models(statistics_training)
sp = remove_before_models(statistics_predict, is_train=False)

In [15]:
from scipy import stats
# Downsampling TODO
for c in st.columns:
    shapiro_test = stats.shapiro(st[c])
    print("column = %s \t statistic = %f pvalue = %f" %(c,shapiro_test.statistic, shapiro_test.pvalue))

column = diff_pezzali 	 statistic = 0.972824 pvalue = 0.000000
column = diff_defensive 	 statistic = 0.991345 pvalue = 0.000003
column = stats_away.c_yellow 	 statistic = 0.941131 pvalue = 0.000000
column = stats_home.c_yellow 	 statistic = 0.937945 pvalue = 0.000000
column = home_team.overall_rank 	 statistic = 0.626416 pvalue = 0.000000
column = away_team.overall_rank 	 statistic = 0.626416 pvalue = 0.000000
column = home_team.ranking 	 statistic = 0.756818 pvalue = 0.000000
column = away_team.ranking 	 statistic = 0.878469 pvalue = 0.000000


In [16]:
for c in sp.columns:
    shapiro_test = stats.shapiro(sp[c])
    print("column = %s \t statistic = %f pvalue = %f" %(c,shapiro_test.statistic, shapiro_test.pvalue))

column = diff_pezzali 	 statistic = 0.952753 pvalue = 0.754677
column = diff_defensive 	 statistic = 0.847514 pvalue = 0.116616
column = stats_away.c_yellow 	 statistic = 0.600399 pvalue = 0.000275
column = stats_home.c_yellow 	 statistic = 0.840044 pvalue = 0.099451
column = home_team.overall_rank 	 statistic = 1.000000 pvalue = 1.000000
column = away_team.overall_rank 	 statistic = 1.000000 pvalue = 1.000000
column = home_team.ranking 	 statistic = 1.000000 pvalue = 1.000000
column = away_team.ranking 	 statistic = 1.000000 pvalue = 1.000000


/anaconda3/envs/thesis/lib/python3.7/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


In [17]:
st = preprocessing.StandardScaler().fit_transform(st)
sp = preprocessing.StandardScaler().fit_transform(sp)
sv_p = SVC(kernel='linear', probability=True)
sv_p.fit(st, target)
print(sv_p.predict(sp))

[ 0 -1 -1  1  0  1  1]


In [18]:
# using scaler on nb
from sklearn.naive_bayes import GaussianNB
sv_p = SVC(kernel='linear', probability=True)
sv_p.fit(st, target)
print(sv_p.predict(sp))

[-1 -1  0  1  0  1  1]


In [21]:
from sklearn import linear_model
ols = linear_model.LogisticRegression()
ols.fit(st, target)
print(ols.predict(sp))

[ 0 -1 -1  1  0  1  1]


In [20]:
from sklearn.ensemble import RandomForestClassifier
# At least two matches en each classification per week
premise_clf = RandomForestClassifier(criterion="entropy", max_features="log2", min_samples_leaf=2).fit(st, target)
print(premise_clf.predict(sp))

[ 0 -1  0  1  0  1  1]
